## Importer les librairies utiles

In [37]:
# Librairie pour la gestion des dates
from datetime import datetime

# Librairies permettant de charger et de manipuler les données
import numpy as np
import pandas as pd

# Methode permetant d'identifier
from sklearn.impute import MissingIndicator

# Méthodes permettants de remplacer des valeurs manquantes avec sklearn
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

# Méthodes permettant de convertir des variables quatégorielles afin de le rendre exploitable à l'anlyse de données
from sklearn.preprocessing import OneHotEncoder

## Charger les données

In [38]:
# Avant le chargement des données on peut définir qu'elles seront les valeurs à remplacer par un NaN
missing_values = ["na","n/a", "--", "-", "#", "##", "N/A", "nan", "?", "no"]
df = pd.read_csv("1.Donnees_brutes/18052022_donnees_CENSUS.csv", delimiter = ",", na_values = missing_values)

In [39]:
df.head()

,age,workclass,education,marital_status,occupation,relationship,race,sex,hours_per_week,native_country,classe,Unnamed: 11,Unnamed: 12
0,41.0,State-gov,Masters,Married-civ-spouse,Exec-managerial,Husband,White,Male,50,United-States,more,NaN,NaN
1,50.0,Private,Masters,Divorced,Sales,Not-in-family,White,Female,50,United-States,more,NaN,NaN
2,24.0,Private,7th-8th,Separated,Machine-op-inspct,Own-child,White,Male,40,United-States,less,NaN,NaN
3,27.0,Without-pay,HS-grad,Never-married,Farming-fishing,Own-child,White,Female,40,United-States,less,NaN,NaN
4,50.0,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,40,United-States,less,NaN,NaN


## Décrire son jeu de données

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             48834 non-null  float64
 1   workclass       48834 non-null  object 
 2   education       48841 non-null  object 
 3   marital_status  48842 non-null  object 
 4   occupation      48840 non-null  object 
 5   relationship    48841 non-null  object 
 6   race            48840 non-null  object 
 7   sex             48840 non-null  object 
 8   hours_per_week  48842 non-null  object 
 9   native_country  48842 non-null  object 
 10  classe          48842 non-null  object 
 11  Unnamed: 11     8 non-null      object 
 12  Unnamed: 12     7 non-null      object 
dtypes: float64(1), object(12)
memory usage: 4.8+ MB


In [41]:
df.tail()

,age,workclass,education,marital_status,occupation,relationship,race,sex,hours_per_week,native_country,classe,Unnamed: 11,Unnamed: 12
48837,57.0,Private,Some-college,Never-married,Protective-serv,Not-in-family,White,Male,40,United-States,less,NaN,NaN
48838,31.0,Private,Bachelors,Married-civ-spouse,NaN,Husband,White,Male,40,United-States,less,NaN,NaN
48839,37.0,Private,Bachelors,Married-civ-spouse,Sales,Husband,White,Male,60,United-States,more,NaN,NaN
48840,20.0,Private,Some-college,Never-married,Other-service,Own-child,White,Female,11,United-States,less,NaN,NaN
48841,20.0,Private,Assoc-acdm,Never-married,Prof-specialty,Own-child,White,Female,20,United-States,less,NaN,NaN


In [42]:
df.shape

(48842, 13)

In [43]:
df["workclass"].nunique()

9

In [44]:
df["workclass"].unique()

array(['State-gov', 'Private', 'Without-pay', 'Self-emp-not-inc',
       'Local-gov', 'Federal-gov', 'Self-emp-inc', 'Never-worked', '19',
       nan], dtype=object)

## Repérer les données manquantes standards

In [45]:
# Voir les données manquantes au global
df.isnull().sum()

age                   8
workclass             8
education             1
marital_status        0
occupation            2
relationship          1
race                  2
sex                   2
hours_per_week        0
native_country        0
classe                0
Unnamed: 11       48834
Unnamed: 12       48835
dtype: int64

In [46]:
education_manquante = df[df['education'].isna()]
education_manquante

,age,workclass,education,marital_status,occupation,relationship,race,sex,hours_per_week,native_country,classe,Unnamed: 11,Unnamed: 12
27277,59.0,Private,NaN,Married-civ-spouse,Adm-clerical,Husband,White,Male,40,United-States,more,NaN,NaN


In [47]:
occupation_manquante = df[df['occupation'].isna()]
occupation_manquante

,age,workclass,education,marital_status,occupation,relationship,race,sex,hours_per_week,native_country,classe,Unnamed: 11,Unnamed: 12
48821,20.0,Private,Some-college,Never-married,NaN,Own-child,White,Male,25,United-States,++,NaN,NaN
48838,31.0,Private,Bachelors,Married-civ-spouse,NaN,Husband,White,Male,40,United-States,less,NaN,NaN


## Repérer les données manquantes non standards ou les erreurs

#### On va analyser la variable "age" car elle contient des valeurs NaN difficilement explicable

In [48]:
age_aberrante = df[df['age'].isna()]
age_aberrante

,age,workclass,education,marital_status,occupation,relationship,race,sex,hours_per_week,native_country,classe,Unnamed: 11,Unnamed: 12
8474,NaN,19,Self-emp-inc,HS-grad,Never-married,Other-service,Own-child,White,Male,6,United-States,less,NaN
15408,NaN,NaN,41,Federal-gov,Masters,Married-civ-spouse,Craft-repair,Husband,White,Male,60,India,more
15409,NaN,NaN,53,Federal-gov,HS-grad,Divorced,Craft-repair,Not-in-family,Black,Male,40,United-States,less
15410,NaN,NaN,61,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Black,Male,40,United-States,less
15411,NaN,NaN,61,Local-gov,Bachelors,Never-married,Prof-specialty,Not-in-family,White,Female,63,United-States,more
15412,NaN,NaN,45,Private,HS-grad,Divorced,Adm-clerical,Unmarried,White,Female,40,United-States,less
15413,NaN,NaN,19,Private,Some-college,Never-married,Other-service,Not-in-family,White,Male,35,United-States,less
15414,NaN,NaN,46,Private,11th,Widowed,Prof-specialty,Unmarried,Black,Female,40,United-States,less


In [49]:
age_aberrante.shape

(8, 13)

On peut constater que les valeurs à NaN sur la colonne "age" correspondent à un décalage que nous réglerons plus tard

#### On va analyser la variable "workclass" car elle contient des valeurs NaN difficilement explicable

In [50]:
workclass_aberrante = df[df['workclass'].isna()]
workclass_aberrante

,age,workclass,education,marital_status,occupation,relationship,race,sex,hours_per_week,native_country,classe,Unnamed: 11,Unnamed: 12
15408,NaN,NaN,41,Federal-gov,Masters,Married-civ-spouse,Craft-repair,Husband,White,Male,60,India,more
15409,NaN,NaN,53,Federal-gov,HS-grad,Divorced,Craft-repair,Not-in-family,Black,Male,40,United-States,less
15410,NaN,NaN,61,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Black,Male,40,United-States,less
15411,NaN,NaN,61,Local-gov,Bachelors,Never-married,Prof-specialty,Not-in-family,White,Female,63,United-States,more
15412,NaN,NaN,45,Private,HS-grad,Divorced,Adm-clerical,Unmarried,White,Female,40,United-States,less
15413,NaN,NaN,19,Private,Some-college,Never-married,Other-service,Not-in-family,White,Male,35,United-States,less
15414,NaN,NaN,46,Private,11th,Widowed,Prof-specialty,Unmarried,Black,Female,40,United-States,less
27936,48.0,NaN,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,50,United-States,more,NaN,NaN


In [51]:
workclass_aberrante.shape

(8, 13)

On peut constater que les valeurs à NaN sur la colonne "workclass" correspondent à un décalage que nous réglerons plus tard

#### On va analyser la variable "hours_per_week" car elle est de forme object alors que la majorité des valeurs sont du type int

In [52]:
df["hours_per_week"].nunique()

99

In [53]:
df["hours_per_week"].unique()

array(['50', '40', '60', '56', '30', '99', '48', '42', '55', '35', '10',
       '65', '91', '58', '45', '5', '24', '70', '52', '38', '25', '37',
       '20', '32', '4', '44', '36', '11', '23', '39', '16', '3', '12',
       '86', '18', '15', '9', '72', '34', '2', '53', '75', '22', '41',
       '17', '43', '14', '8', '68', '46', '80', '6', '28', '27', '49',
       '26', '13', '77', '51', '84', '7', '47', '98', '59', '67', '21',
       '33', '1', '90', '54', '89', '85', '31', '66', '74', '19', '57',
       '62', '29', '88', '95', '78', '61', '64', 'Male', '63', '81', '82',
       '87', 'White', 'Black', '73', '76', '92', '96', '94', '79', '97',
       '69'], dtype=object)

In [54]:
hours_per_week_aberrante = df[~df['hours_per_week'].str.isnumeric()]
hours_per_week_aberrante

,age,workclass,education,marital_status,occupation,relationship,race,sex,hours_per_week,native_country,classe,Unnamed: 11,Unnamed: 12
8474,NaN,19,Self-emp-inc,HS-grad,Never-married,Other-service,Own-child,White,Male,6,United-States,less,NaN
15408,NaN,NaN,41,Federal-gov,Masters,Married-civ-spouse,Craft-repair,Husband,White,Male,60,India,more
15409,NaN,NaN,53,Federal-gov,HS-grad,Divorced,Craft-repair,Not-in-family,Black,Male,40,United-States,less
15410,NaN,NaN,61,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Black,Male,40,United-States,less
15411,NaN,NaN,61,Local-gov,Bachelors,Never-married,Prof-specialty,Not-in-family,White,Female,63,United-States,more
15412,NaN,NaN,45,Private,HS-grad,Divorced,Adm-clerical,Unmarried,White,Female,40,United-States,less
15413,NaN,NaN,19,Private,Some-college,Never-married,Other-service,Not-in-family,White,Male,35,United-States,less
15414,NaN,NaN,46,Private,11th,Widowed,Prof-specialty,Unmarried,Black,Female,40,United-States,less


In [55]:
hours_per_week_aberrante.shape

(8, 13)

Nous pouvons constater qu'il y a 8 lignes qui sont décalées vers la droite 

#### On va analyser la variable "classe" 

In [56]:
df['classe'].nunique()

8

In [57]:
df['classe'].unique()

array(['more', 'less', 'United-States', '60', '40', '63', '35', '++'],
      dtype=object)

In [58]:
df['classe'].value_counts()

less             37148
more             11685
40                   4
United-States        1
60                   1
63                   1
35                   1
++                   1
Name: classe, dtype: int64

On peut constater  que la classe possède 9 valeurs qui ne sont pas coherentes avec le reste. On en déduit aussi que seule la valeur "++" est non stantard sans décalage de colonne

In [59]:
df.dtypes

age               float64
workclass          object
education          object
marital_status     object
occupation         object
relationship       object
race               object
sex                object
hours_per_week     object
native_country     object
classe             object
Unnamed: 11        object
Unnamed: 12        object
dtype: object

## Nettoyage des données

In [60]:
df.shape

(48842, 13)

In [61]:
df.isnull().sum()

age                   8
workclass             8
education             1
marital_status        0
occupation            2
relationship          1
race                  2
sex                   2
hours_per_week        0
native_country        0
classe                0
Unnamed: 11       48834
Unnamed: 12       48835
dtype: int64

In [62]:
columns = df.columns

#### On récupére les lignes décalées et avec les valeurs manquantes

In [63]:
df_aberrants = pd.DataFrame()
for column in df.columns:
    if column not in ["Unnamed: 11", "Unnamed: 12"]:
        df_aberrants = pd.concat([df_aberrants, df[df[column].isna()]])

df_aberrants.drop_duplicates(inplace=True)
df_aberrants.shape

(17, 13)

#### On exporte nos valeurs manquantes ou aberrantes

In [64]:
curDT = datetime.now()
dd = curDT.strftime("%d%m%Y")
df_aberrants.to_csv(f"./2.Donnees_nettoyees/{dd}_donnees_aberrantes.csv")

#### On supprime les colonnes aberrantes

In [65]:
df2 = df.copy()
df2.drop(columns=["Unnamed: 11", "Unnamed: 12"], inplace=True)
df2.columns

Index(['age', 'workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'hours_per_week', 'native_country',
       'classe'],
      dtype='object')

#### On supprime les lignes aberrantes et manquantes

In [66]:
df2.shape

(48842, 11)

In [67]:
df2.dropna(inplace=True)

#### On vérifie les données

In [68]:
df2.shape

(48825, 11)

In [69]:
df2["classe"].unique()

array(['more', 'less'], dtype=object)

In [70]:
df2.isnull().sum()

age               0
workclass         0
education         0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
hours_per_week    0
native_country    0
classe            0
dtype: int64

#### On exporte le nouveau DataFrame dans nos données chargées 

In [71]:
curDT = datetime.now()
dd = curDT.strftime("%d%m%Y")
df2.to_csv(f"./3.Donnees_chargees/{dd}_donnees_CENSUS.csv", index=False)